In [5]:
# read tiny shakespeare
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()


In [7]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [15]:
# get disinct characters
chars = sorted(set(text))
vocab_size = len(chars)
vocab_size

65

In [21]:
# maps from char to int and inverse
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda x: [stoi[c] for c in x]
decode = lambda x: "".join([itos[i] for i in x])
test_str = text[:10]
print(test_str)
print(encode(test_str))
print(decode(encode(test_str)))

First Citi
[18, 47, 56, 57, 58, 1, 15, 47, 58, 47]
First Citi


In [23]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch 
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [25]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [29]:
# for seq data we now have a seq len paramter
# in addition to the batch size
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [33]:
# this illustrates what the network will do during training
# we have a context window of lenght l and try to predict the l+1th token
# this is important for inference later on since we need to be able 
# to handle variable length inputs
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [45]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,)) # grab batch size number of starting positions
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

## A mathematical detour for self attention

In [211]:
torch.manual_seed(1337)
B,T,C = 4,8,2  #batch time channels
x = torch.randn(B,T,C)


#if we are at the t'th step in a batch we can use the info of all the tokens that came
#before us. The simplest way to do this is just an average over the previous tokens.

xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #(t,C)
        xbow[b,t] = torch.mean(xprev,0)



In [213]:
# now we want to make it fast
m = torch.ones((T,T))
m = torch.tril(m)
m = m/m.sum(1,keepdim=True)
print(m)
# B X T X T @ B X T X C 
#A,m@A
torch.allclose(m @ x,xbow)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


False

In [217]:
# now the same idea but we use softmax instead
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei,dim=-1) # exp(0) = 1, exp(-inf) = 0

In [233]:
# full self attention
torch.manual_seed(1337)
B,T,C = 4,8,32  #batch time channels
x = torch.randn(B,T,C)

# single head of self attention
# each token is allowed to communicate with every other token by
# 1 sending a query vector
# 2 representing itself as a key
# we simply take the dot product to measure how much a tokens query
#  aligns with another tokens key. 
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value  = nn.Linear(C, head_size, bias=False)
k = key(x) # B X T X H
q = query(x) # B X T X H
# this magic number keeps the output having unit variance
# otherwise in softmax the distribution will collapse
# to a few or even one point
wei = q @ k.transpose(-2,-1) * head_size**-0.5 # B X T X T 

# here we make a lower triangular mask so only interections with past
# tokens are counted towards anything. Finally we do a softmax to turn
# them into "probabilities"
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei,dim=-1) # exp(0) = 1, exp(-inf) = 0
v = value(x) # this seems dumb and arbitrary
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [317]:
class Head(nn.Module):

    def __init__(self, block_size, n_embd, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size,bias=False)
        self.query = nn.Linear(n_embd,head_size,bias=False)
        self.value = nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))
        self.head_size = head_size
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * self.head_size**-0.5
        tril = torch.tril(torch.ones(T,T))
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float("-inf"))
        wei = F.softmax(wei,dim=-1) # exp(0) = 1, exp(-inf) = 0
        wei = self.dropout(wei)
        v = self.value(x) # this seems dumb and arbitrary
        out = wei @ v
        return out 



In [277]:
# we can stack a bunch of heads and run them in parallel
# in our machine learning fantasies we tell ourselves each
# one is learning some different part of the task 
class MultiHeadAttention(nn.Module):

    def __init__(self, n_heads, block_size, n_embd, head_size,dropout=0.1):
        super().__init__()
        self.heads = nn.ModuleList([Head(block_size, n_embd, head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(n_embd,n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads],dim=-1)
        out = self.dropout(self.proj(out)) # project back to embedding size but in an epic machine learning way
        return out 

In [305]:
# we shove a feedfoward network at the end of each block 
# to do postprocessing on the output of the attention layers
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [307]:
class Block(nn.Module):

    def __init__(self, n_heads, block_size, n_embd):
        super().__init__()
        head_size = n_embd//n_heads
        self.sa = MultiHeadAttention(n_heads, block_size, n_embd, head_size)
        self.ff = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self,x):
        # add residual connections to help backprop gradients
        # chain rule will flow all of the gradient thru the left branch
        # add layer norm which normalizes the last dimension in the input data to be N(0,1)
        x = x + self.sa(self.ln1(x)) 
        x = x + self.ff(self.ln2(x))
        return x


In [351]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size, block_size, n_embd, n_heads, n_layers):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_heads, block_size, n_embd) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            print(F.softmax(logits[0]))
            print(targets[0])
            loss = F.cross_entropy(logits, targets)
            print(loss)
            print()

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # batch dimension is kind of strange but would let you
        # generate from many different starting sentences
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # with the probabilities we can sample directly the 
            # next character as a multinomial
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [353]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_heads = 4
n_layers = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

model = BigramLanguageModel(vocab_size, block_size, n_embd,  n_heads, n_layers)

In [ ]:
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.209729 M parameters
tensor([0.0557, 0.0206, 0.0061, 0.0336, 0.0169, 0.0135, 0.0062, 0.0078, 0.0101,
        0.0093, 0.0080, 0.0216, 0.0253, 0.0020, 0.0191, 0.0116, 0.0120, 0.0064,
        0.0104, 0.0243, 0.0056, 0.0098, 0.0102, 0.0083, 0.0137, 0.0372, 0.0110,
        0.0094, 0.0102, 0.0046, 0.0415, 0.0102, 0.0083, 0.0231, 0.0122, 0.0119,
        0.0188, 0.0184, 0.0141, 0.0133, 0.0085, 0.0098, 0.0088, 0.0116, 0.0209,
        0.0115, 0.0163, 0.0116, 0.0116, 0.0195, 0.0264, 0.0104, 0.0235, 0.0054,
        0.0113, 0.0136, 0.0188, 0.0094, 0.0128, 0.0234, 0.0457, 0.0074, 0.0286,
        0.0109, 0.0102])
tensor(6)
tensor(4.3815)

tensor([0.0434, 0.0188, 0.0067, 0.0166, 0.0093, 0.0172, 0.0037, 0.0090, 0.0151,
        0.0088, 0.0086, 0.0108, 0.0395, 0.0096, 0.0189, 0.0075, 0.0117, 0.0090,
        0.0134, 0.0191, 0.0067, 0.0186, 0.0203, 0.0049, 0.0180, 0.0483, 0.0133,
        0.0111, 0.0088, 0.0076, 0.0173, 0.0101, 0.0074, 0.0239, 0.0165, 0.0169,
        0.0135, 0.0195, 0.0162, 0.0108, 0.0089,

C:\Users\blake\AppData\Local\Temp\ipykernel_21532\810786751.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(logits[0]))


tensor([0.0542, 0.0548, 0.0078, 0.0226, 0.0050, 0.0160, 0.0059, 0.0073, 0.0184,
        0.0056, 0.0077, 0.0082, 0.0259, 0.0160, 0.0128, 0.0045, 0.0069, 0.0213,
        0.0127, 0.0117, 0.0185, 0.0069, 0.0309, 0.0054, 0.0126, 0.0282, 0.0240,
        0.0080, 0.0058, 0.0135, 0.0176, 0.0035, 0.0026, 0.0121, 0.0191, 0.0105,
        0.0203, 0.0132, 0.0115, 0.0097, 0.0128, 0.0237, 0.0238, 0.0074, 0.0316,
        0.0082, 0.0107, 0.0171, 0.0281, 0.0123, 0.0096, 0.0085, 0.0216, 0.0223,
        0.0069, 0.0062, 0.0286, 0.0065, 0.0105, 0.0124, 0.0207, 0.0044, 0.0412,
        0.0125, 0.0136])
tensor(5)
tensor(4.4088)

tensor([0.0557, 0.0206, 0.0061, 0.0336, 0.0169, 0.0135, 0.0062, 0.0078, 0.0101,
        0.0093, 0.0080, 0.0216, 0.0253, 0.0020, 0.0191, 0.0116, 0.0120, 0.0064,
        0.0104, 0.0243, 0.0056, 0.0098, 0.0102, 0.0083, 0.0137, 0.0372, 0.0110,
        0.0094, 0.0102, 0.0046, 0.0415, 0.0102, 0.0083, 0.0231, 0.0122, 0.0119,
        0.0188, 0.0184, 0.0141, 0.0133, 0.0085, 0.0098, 0.0088, 0.011